In [1]:
import pandas as pd
import re
import os
import matplotlib.pyplot as plt

In [3]:
raw_data_files = os.listdir("../ingestion_service/data/raw_data")
csv_files = [f for f in raw_data_files if re.search(r'.*\.csv$', f)]
csv_files

['esg_raw_data_on_corporations_myu_2.csv',
 'esg_raw_data_on_corporations_myu_3.csv',
 'esg_raw_data_on_corporations_7.csv',
 'esg_raw_data_on_corporations_6.csv',
 'esg_raw_data_on_corporations_myu_4.csv',
 'esg_raw_data_on_corporations_myu_5.csv',
 'esg_raw_data_on_corporations_1.csv']

In [7]:
dfs = []
number_of_rows = 0
for csv in csv_files:
    print(f"Reading {csv}")
    df_chunk = pd.read_csv(f"../ingestion_service/data/raw_data/{csv}", sep='|')
    print(df_chunk.shape)
    number_of_rows += df_chunk.shape[0]
    # Optional preprocessing here
    # df_chunk = df_chunk[['column1', 'column2']]  # example
    
    dfs.append(df_chunk)

print("total number of rows in raw data: ", number_of_rows)
df = pd.concat(dfs, ignore_index=True)

Reading esg_raw_data_on_corporations_myu_2.csv
(1322420, 15)
Reading esg_raw_data_on_corporations_myu_3.csv
(1024697, 15)
Reading esg_raw_data_on_corporations_7.csv
(86371, 15)
Reading esg_raw_data_on_corporations_6.csv
(257603, 15)
Reading esg_raw_data_on_corporations_myu_4.csv
(812742, 15)
Reading esg_raw_data_on_corporations_myu_5.csv
(522838, 15)
Reading esg_raw_data_on_corporations_1.csv
(2764252, 15)
total number of rows in raw data:  6790923


In [12]:
def add_industry_matching(esg_df):
    try:
        companies_df = pd.read_csv('../ingestion_service/data/industry.csv')

        companies_df = companies_df.rename(columns={'Industry': 'industry_name'})
        companies_df.drop(columns=['Company Name'], inplace=True)

        res_df = pd.merge(esg_df, companies_df, how="left", on=['perm_id'])
        return res_df
        
    except Exception as e:
        raise Exception(f'Exception while running industry matching {e}')

In [14]:
df = add_industry_matching(df)

In [15]:
df.head()

,company_name,perm_id,data_type,disclosure,metric_description,metric_name,metric_unit,metric_value,metric_year,nb_points_of_observations,metric_period,provider_name,reported_date,pillar,headquarter_country,industry_name
0,Abdullah Saad Mohammed Abo Moati for Bookstore...,5052962521,int,REPORTED,Does the company have a formal grievance repor...,GRIEVANCE_REPORTING_PROCESS,Yes/No,0.00,2022-12-31,331,NaN,Clarity AI,2022-12-31 00:00:00,S,Saudi Arabia,Chemicals
1,Abdullah Saad Mohammed Abo Moati for Bookstore...,5052962521,int,CALCULATED,Does the company have appropriate communicatio...,IMPROVEMENT_TOOLS_BUSINESS_ETHICS,Yes/No,0.00,2019-12-31,331,NaN,Clarity AI,2019-09-27 14:29:37,S,Saudi Arabia,Chemicals
2,Abdullah Saad Mohammed Abo Moati for Bookstore...,5052962521,float,ESTIMATED,External cost of indirect emissions of polluta...,AIRPOLLUTANTS_INDIRECT,USD (000),193067.06,2021-12-31,331,NaN,Clarity AI,NaN,E,Saudi Arabia,Chemicals
3,Abdullah Saad Mohammed Abo Moati for Bookstore...,5052962521,int,REPORTED,Does the company have appropriate communicatio...,IMPROVEMENT_TOOLS_BUSINESS_ETHICS,Yes/No,0.00,2024-12-31,331,NaN,Clarity AI,2024-12-31 00:00:00,S,Saudi Arabia,Chemicals
4,Abdullah Saad Mohammed Abo Moati for Bookstore...,5052962521,float,REPORTED,The average overall attendance percentage of b...,BOARDMEETINGATTENDANCEAVG,%,100.00,2022-12-31,331,NaN,Clarity AI,2022-12-31 00:00:00,G,Saudi Arabia,Chemicals


# Rename Eurofidai Metrics name

In [18]:
# Filter numerical metrics
int_metric_names = df[df['data_type'] == 'int']['metric_name'].unique().tolist()
metric_list = df['metric_name'].unique().tolist()
numerical_metric = df[df['metric_unit'] != 'Yes/No']
metrics_info = numerical_metric[['metric_name', 'metric_description', 'metric_unit', 'data_type', 'pillar']].drop_duplicates()
metrics_info.sort_values(by=['pillar', 'metric_name'])

,metric_name,metric_description,metric_unit,data_type,pillar
1029,AIRPOLLUTANTS_DIRECT,External cost of pollutants released to air by...,USD (000),float,E
2,AIRPOLLUTANTS_INDIRECT,External cost of indirect emissions of polluta...,USD (000),float,E
1958,ANALYTICWASTERECYCLINGRATIO,Total recycled and reused waste produced in to...,%,float,E
54,CO2DIRECTSCOPE1,Scope 1 emissions that occur within a company’...,Tons CO2e,float,E
20,CO2INDIRECTSCOPE2,Scope 2 emissions that result from the generat...,Tons CO2e,float,E
39,CO2INDIRECTSCOPE3,Scope 3 emissions that occur in the value chai...,Tons CO2e,float,E
4180543,CO2_NO_EQUIVALENTS,The estimated total CO2 emission in tons (with...,Tons CO2,float,E
115882,ELECTRICITYPURCHASED,Electricity purchased in gigajoules.,GJ,float,E
257,EMS_CERTIFIED_PCT,The percentage of company sites or subsidiarie...,%,float,E
511474,ENERGYPURCHASEDDIRECT,Direct energy purchased in gigajoules.,GJ,float,E


In [20]:
# export metrics to csv
metrics_info.sort_values(by=['pillar', 'metric_name']).to_csv('../ingestion_service/data/numerical_metric_info.csv', index=False)

In [22]:
# manual upload updated metric name
updated_metric_info = pd.read_csv('../ingestion_service/data/numerical_metric_info_update.csv')
updated_metric_info

,metric_name,metric_name_update,metric_description,metric_unit,data_type,pillar
0,AIRPOLLUTANTS_DIRECT,Direct air pollution cost,External cost of pollutants released to air by...,USD (000),float,E
1,AIRPOLLUTANTS_INDIRECT,Indirect air pollution cost,External cost of indirect emissions of polluta...,USD (000),float,E
2,ANALYTICWASTERECYCLINGRATIO,Recycled and reused waste %,Total recycled and reused waste produced in to...,%,float,E
3,CO2DIRECTSCOPE1,Direct CO2 emissions scope 1,Scope 1 emissions that occur within a company’...,Tons CO2e,float,E
4,CO2INDIRECTSCOPE2,Indirect CO2 emissions scope 2,Scope 2 emissions that result from the generat...,Tons CO2e,float,E
5,CO2INDIRECTSCOPE3,Indirect CO2 emissions scope 3,Scope 3 emissions that occur in the value chai...,Tons CO2e,float,E
6,CO2_NO_EQUIVALENTS,Estimated total CO2 emissions,The estimated total CO2 emission in tons (with...,Tons CO2,float,E
7,ELECTRICITYPURCHASED,Purchased electricity,Electricity purchased in gigajoules.,GJ,float,E
8,EMS_CERTIFIED_PCT,EMS certified sites %,The percentage of company sites or subsidiarie...,%,float,E
9,ENERGYPURCHASEDDIRECT,Direct energy purchased,Direct energy purchased in gigajoules.,GJ,float,E


## Semiconductor Industry for pca model (both numerical and categorical metrics present in csv, may handle it in pca model)

In [25]:
semiconductor_industry = df[df['industry_name'] == 'Semiconductors']
semiconductor_industry

,company_name,perm_id,data_type,disclosure,metric_description,metric_name,metric_unit,metric_value,metric_year,nb_points_of_observations,metric_period,provider_name,reported_date,pillar,headquarter_country,industry_name
8637,Naseej for Technology Company SJSC,4296741969,int,REPORTED,"Does the company describe, claim to have or me...",POLICY_FREEDOMOF_ASSOCIATION,Yes/No,0.00,2021-12-31,274,NaN,Clarity AI,2021-12-31 00:00:00,S,NaN,Semiconductors
8638,Naseej for Technology Company SJSC,4296741969,int,CALCULATED,Does the company have a policy to avoid the us...,POLICY_FORCED_LABOR,Yes/No,0.00,2020-12-31,274,NaN,Clarity AI,2020-12-31 00:00:00,S,NaN,Semiconductors
8639,Naseej for Technology Company SJSC,4296741969,int,CALCULATED,Is the senior executive's compensation linked ...,ANALYTICCSR_COMP_INCENTIVES,Yes/No,0.00,2019-12-31,274,NaN,Clarity AI,2019-12-31 00:00:00,G,NaN,Semiconductors
8640,Naseej for Technology Company SJSC,4296741969,float,ESTIMATED,Average amount of water consumed and reclaimed...,WATER_USE_PAI_M10,Cubic meters / million EUR of revenue of inves...,0.00,2024-12-31,274,NaN,Clarity AI,2024-12-20 00:00:00,E,NaN,Semiconductors
8641,Naseej for Technology Company SJSC,4296741969,int,CALCULATED,Are all shares of the company providing equal ...,ANALYTIC_VOTING_RIGHTS,Yes/No,0.00,2020-12-31,274,NaN,Clarity AI,2020-12-31 00:00:00,G,NaN,Semiconductors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778830,Seoul Semiconductor Co Ltd,4295882677,float,ESTIMATED,Scope 2 emissions that result from the generat...,CO2INDIRECTSCOPE2,Tons CO2e,41959.73,2022-12-31,166,NaN,Clarity AI,NaN,E,"Korea, Rep.",Semiconductors
6778835,Seoul Semiconductor Co Ltd,4295882677,int,CALCULATED,Has the company established a program or an in...,ANIMAL_TESTING_REDUCTION,Yes/No,0.00,2019-12-31,166,NaN,Clarity AI,2019-12-31 00:00:00,S,"Korea, Rep.",Semiconductors
6778849,Seoul Semiconductor Co Ltd,4295882677,int,CALCULATED,Does the company report on making proactive en...,ENV_INVESTMENTS,Yes/No,0.00,2019-12-31,166,NaN,Clarity AI,2019-12-31 00:00:00,E,"Korea, Rep.",Semiconductors
6778854,Seoul Semiconductor Co Ltd,4295882677,float,ESTIMATED,Amount of Volatile Organic Compounds (VOC) emi...,VOCEMISSIONS,Tons of VOC,25.78,2019-12-31,166,NaN,Clarity AI,NaN,E,"Korea, Rep.",Semiconductors


In [27]:
companies_in_semiconductors = semiconductor_industry['perm_id'].nunique()
companies_in_semiconductors

859

In [29]:
# sort by perm_id, then year
pivot_df = semiconductor_industry.pivot(index=['perm_id', 'metric_year'], columns='metric_name', values='metric_value')
pivot_df

metric_name             AIRPOLLUTANTS_DIRECT  AIRPOLLUTANTS_INDIRECT  \
perm_id    metric_year                                                 
4295857321 2016-12-31                    NaN                     NaN   
           2017-12-31                    NaN                     NaN   
           2018-12-31                    NaN                 5423.10   
           2019-12-31                    NaN                 2044.09   
           2020-12-31                    NaN                 2140.60   
...                                      ...                     ...   
5083349161 2023-12-31                    NaN                     NaN   
5084952432 2019-12-31                    NaN                     NaN   
           2020-12-31                    NaN                     NaN   
           2021-12-31                    NaN                     NaN   
           2022-12-31                    NaN                     NaN   

metric_name             ANALYTICAUDITCOMMIND  ANALYTICBOARDFEMALE  \
perm_id    metric_year                                              
4295857321 2016-12-31                    NaN                  NaN   
           2017-12-31                    NaN                  NaN   
           2018-12-31                    NaN                  NaN   
           2019-12-31                    NaN                  NaN   
           2020-12-31                    NaN                  NaN   
...                                      ...                  ...   
5083349161 2023-12-31                    NaN                  NaN   
5084952432 2019-12-31                    NaN                  NaN   
           2020-12-31                    NaN                  NaN   
           2021-12-31                    NaN                  NaN   
           2022-12-31                    NaN                  NaN   

metric_name             ANALYTICCEO_CHAIRMAN_SEPARATION  ANALYTICCOMPCOMMIND  \
perm_id    metric_year                                                         
4295857321 2016-12-31                               NaN                  NaN   
           2017-12-31                               NaN                  NaN   
           2018-12-31                               NaN                  NaN   
           2019-12-31                               NaN                  NaN   
           2020-12-31                               NaN                  NaN   
...                                                 ...                  ...   
5083349161 2023-12-31                               NaN                  NaN   
5084952432 2019-12-31                               NaN                  NaN   
           2020-12-31                               NaN                  NaN   
           2021-12-31                               NaN                  NaN   
           2022-12-31                               NaN                  NaN   

metric_name             ANALYTICCSR_COMP_INCENTIVES  \
perm_id    metric_year                                
4295857321 2016-12-31                           NaN   
           2017-12-31                           NaN   
           2018-12-31                           NaN   
           2019-12-31                           0.0   
           2020-12-31                           0.0   
...                                             ...   
5083349161 2023-12-31                           NaN   
5084952432 2019-12-31                           NaN   
           2020-12-31                           NaN   
           2021-12-31                           NaN   
           2022-12-31                           NaN   

metric_name             ANALYTICEMPLOYMENTCREATION  ANALYTICINDEPBOARD  \
perm_id    metric_year                                                   
4295857321 2016-12-31                         5.56                 NaN   
           2017-12-31                         5.26                 NaN   
           2018-12-31                         0.00                 NaN   
           2019-12-31                 

In [34]:
pivot_df.to_csv('semiconductor.csv')

In [36]:
# by year first, then by perm_id
pivot_df1 = semiconductor_industry.pivot(index=['metric_year', 'perm_id'], columns='metric_name', values='metric_value')
pivot_df1

metric_name             AIRPOLLUTANTS_DIRECT  AIRPOLLUTANTS_INDIRECT  \
metric_year perm_id                                                    
2016-12-31  4295857321                   NaN                     NaN   
            4295857863                   NaN                     NaN   
            4295858147                  5.31                     NaN   
            4295859475                 17.33                     NaN   
            4295859530                   NaN                     NaN   
...                                      ...                     ...   
2024-12-31  5080529117                   NaN                     NaN   
            5081079864                   NaN                     NaN   
            5081654561                   NaN                     NaN   
            5082156409                   NaN                     NaN   
            5082432234                   NaN                     NaN   

metric_name             ANALYTICAUDITCOMMIND  ANALYTICBOARDFEMALE  \
metric_year perm_id                                                 
2016-12-31  4295857321                   NaN                  NaN   
            4295857863                   NaN                  NaN   
            4295858147                   NaN                  NaN   
            4295859475                   NaN                  NaN   
            4295859530                   NaN                  NaN   
...                                      ...                  ...   
2024-12-31  5080529117                   NaN                  NaN   
            5081079864                   NaN                  NaN   
            5081654561                   NaN                  NaN   
            5082156409                   NaN                  NaN   
            5082432234                   NaN                  NaN   

metric_name             ANALYTICCEO_CHAIRMAN_SEPARATION  ANALYTICCOMPCOMMIND  \
metric_year perm_id                                                            
2016-12-31  4295857321                              NaN                  NaN   
            4295857863                              NaN                  NaN   
            4295858147                              NaN                  NaN   
            4295859475                              NaN                  NaN   
            4295859530                              NaN                  NaN   
...                                                 ...                  ...   
2024-12-31  5080529117                              NaN                  NaN   
            5081079864                              NaN                  NaN   
            5081654561                              NaN                  NaN   
            5082156409                              NaN                  NaN   
            5082432234                              NaN                  NaN   

metric_name             ANALYTICCSR_COMP_INCENTIVES  \
metric_year perm_id                                   
2016-12-31  4295857321                          NaN   
            4295857863                          NaN   
            4295858147                          NaN   
            4295859475                          NaN   
            4295859530                          NaN   
...                                             ...   
2024-12-31  5080529117                          NaN   
            5081079864                          NaN   
            5081654561                          NaN   
            5082156409                          NaN   
            5082432234                          NaN   

metric_name             ANALYTICEMPLOYMENTCREATION  ANALYTICINDEPBOARD  \
metric_year perm_id                                                      
2016-12-31  4295857321                        5.56                 NaN   
            4295857863                         NaN                 NaN   
            4295858147                         NaN                 NaN   
            4295859475                

In [38]:
pivot_df1.to_csv('semiconductor_by_year.csv')

In [42]:
set(df['metric_name'].unique().tolist()) - set(pivot_df.columns.tolist())

{'CO2_NO_EQUIVALENTS', 'ENERGYPURCHASEDDIRECT'}

In [44]:
#missing two metrics in semiconductors industry
semiconductor_industry[semiconductor_industry['metric_name'] == 'ENERGYPURCHASEDDIRECT']
semiconductor_industry[semiconductor_industry['metric_name'] == 'CO2_NO_EQUIVALENTS']

,company_name,perm_id,data_type,disclosure,metric_description,metric_name,metric_unit,metric_value,metric_year,nb_points_of_observations,metric_period,provider_name,reported_date,pillar,headquarter_country,industry_name


,company_name,perm_id,data_type,disclosure,metric_description,metric_name,metric_unit,metric_value,metric_year,nb_points_of_observations,metric_period,provider_name,reported_date,pillar,headquarter_country,industry_name


## Semiconductor Industry map metrics under categories IFRS S2 for sasb
### This part will export a csv and we match metrics to categories manually in csv


In [46]:
semiconductor_industry = df[df['industry_name'] == 'Semiconductors']

In [48]:
semiconductor_numerical_metrics = semiconductor_industry[semiconductor_industry['metric_unit'] != 'Yes/No']

In [50]:
semiconductor_numerical_metrics_unique = semiconductor_numerical_metrics[['metric_name', 'metric_description', 'metric_unit', 'data_type', 'pillar']].drop_duplicates()

In [52]:
semiconductor_numerical_metrics_df = semiconductor_numerical_metrics_unique.merge(
    updated_metric_info[['metric_name', 'metric_name_update']],
    on='metric_name',
    how='left'
)
semiconductor_numerical_metrics_df['category'] = ''

In [54]:
semiconductor_numerical_metrics_df.sort_values(by=['pillar', 'metric_name']).to_csv('semiconductor_metric_info.csv', index=False)

## Biotechnology & Pharmaceuticals for pca

In [57]:
bp_industry = df[df['industry_name'] == 'Biotechnology & Pharmaceuticals']
bp_industry

,company_name,perm_id,data_type,disclosure,metric_description,metric_name,metric_unit,metric_value,metric_year,nb_points_of_observations,metric_period,provider_name,reported_date,pillar,headquarter_country,industry_name
21820,Rohto Pharmaceutical Co Ltd,4295877894,float,ESTIMATED,Total amount of waste produced in tons.,WASTETOTAL,Tons,8249.13,2018-12-31,251,NaN,Clarity AI,NaN,E,Japan,Biotechnology & Pharmaceuticals
21821,Rohto Pharmaceutical Co Ltd,4295877894,int,REPORTED,Does the company have a policy to improve emis...,POLICY_EMISSIONS,Yes/No,1.00,2024-12-31,251,NaN,Clarity AI,2024-12-31 00:00:00,E,Japan,Biotechnology & Pharmaceuticals
21825,Rohto Pharmaceutical Co Ltd,4295877894,float,ESTIMATED,Total energy consumed by a company within its ...,ENERGYUSETOTAL,GJ,185727.92,2016-12-31,251,NaN,Clarity AI,NaN,E,Japan,Biotechnology & Pharmaceuticals
21831,Rohto Pharmaceutical Co Ltd,4295877894,float,ESTIMATED,Total amount of NOx emissions emitted in tons.,NOXEMISSIONS,Tons of NOx,15.48,2018-12-31,251,NaN,Clarity AI,NaN,E,Japan,Biotechnology & Pharmaceuticals
21844,Rohto Pharmaceutical Co Ltd,4295877894,int,CALCULATED,Does the company report on its impact on biodi...,BIODIVERSITY_IMPACT_REDUCTION,Yes/No,0.00,2019-12-31,251,NaN,Clarity AI,2019-09-27 14:29:37,E,Japan,Biotechnology & Pharmaceuticals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790787,Flexion Therapeutics Inc,4297906483,int,REPORTED,Does the company report on initiatives to redu...,N_OXS_OX_EMISSIONS_REDUCTION,Yes/No,0.00,2020-12-31,134,NaN,Clarity AI,2020-12-31 00:00:00,E,United States,Biotechnology & Pharmaceuticals
6790788,Flexion Therapeutics Inc,4297906483,float,ESTIMATED,Amount of Volatile Organic Compounds (VOC) emi...,VOCEMISSIONS,Tons of VOC,2.03,2020-12-31,134,NaN,Clarity AI,NaN,E,United States,Biotechnology & Pharmaceuticals
6790789,Flexion Therapeutics Inc,4297906483,float,ESTIMATED,Scope 1 emissions that occur within a company’...,CO2DIRECTSCOPE1,Tons CO2e,9.52,2017-12-31,134,NaN,Clarity AI,NaN,E,United States,Biotechnology & Pharmaceuticals
6790790,Flexion Therapeutics Inc,4297906483,int,CALCULATED,Does the company report or show to use human r...,HUMAN_RIGHTS_CONTRACTOR,Yes/No,0.00,2019-12-31,134,NaN,Clarity AI,2019-12-31 00:00:00,S,United States,Biotechnology & Pharmaceuticals


In [59]:
companies_in_bp = bp_industry['perm_id'].nunique()
companies_in_bp

3157

In [61]:
pivot_df = bp_industry.pivot(index=['perm_id', 'metric_year'], columns='metric_name', values='metric_value')
pivot_df.to_csv('biotech.csv')

In [63]:
pivot_df1 = bp_industry.pivot(index=['metric_year', 'perm_id'], columns='metric_name', values='metric_value')
pivot_df1.to_csv('biotech_by_year.csv')

In [65]:
set(df['metric_name'].unique().tolist()) - set(pivot_df.columns.tolist())

{'CO2_NO_EQUIVALENTS', 'ENERGYPURCHASEDDIRECT'}

## Biotechnology & Pharmaceuticals Industry map metrics under categories IFRS S2
### This part will export a csv and we match metrics to categories manually in csv

In [70]:
bio_pharm_industry = df[df['industry_name'] == 'Biotechnology & Pharmaceuticals']
bio_pharm_numerical_metrics = bio_pharm_industry[bio_pharm_industry['metric_unit'] != 'Yes/No']
bio_pharm_numerical_metrics_unique = bio_pharm_numerical_metrics[['metric_name', 'metric_description', 'metric_unit', 'data_type', 'pillar']].drop_duplicates()
bio_pharm_numerical_metrics_df = bio_pharm_numerical_metrics_unique.merge(
    updated_metric_info[['metric_name', 'metric_name_update']],
    on='metric_name',
    how='left'
)
bio_pharm_numerical_metrics_df['category'] = ''
bio_pharm_numerical_metrics_df.sort_values(by=['pillar', 'metric_name']).to_csv('bio_pharm_metric_info.csv', index=False)


## Software & IT services Industry map metrics under categories IFRS S2
### This part will export a csv and we match metrics to categories manually in csv

In [73]:
software_it_industry = df[df['industry_name'] == 'Software & IT Services']
software_it_numerical_metrics = software_it_industry[software_it_industry['metric_unit'] != 'Yes/No']
software_it_numerical_metrics_unique = software_it_numerical_metrics[['metric_name', 'metric_description', 'metric_unit', 'data_type', 'pillar']].drop_duplicates()
software_it_numerical_metrics_df = software_it_numerical_metrics_unique.merge(
    updated_metric_info[['metric_name', 'metric_name_update']],
    on='metric_name',
    how='left'
)
software_it_numerical_metrics_df['category'] = ''
software_it_numerical_metrics_df.sort_values(by=['pillar', 'metric_name']).to_csv('software_it_metric_info.csv', index=False)


## Industrial Machinery & Goods	 Industry map metrics under categories IFRS S2
### This part will export a csv and we match metrics to categories manually in csv

In [75]:
machinery_goods_industry = df[df['industry_name'] == 'Industrial Machinery & Goods']
machinery_goods_numerical_metrics = machinery_goods_industry[machinery_goods_industry['metric_unit'] != 'Yes/No']
machinery_goods_numerical_metrics_unique = machinery_goods_numerical_metrics[['metric_name', 'metric_description', 'metric_unit', 'data_type', 'pillar']].drop_duplicates()
machinery_goods_numerical_metrics_df = machinery_goods_numerical_metrics_unique.merge(
    updated_metric_info[['metric_name', 'metric_name_update']],
    on='metric_name',
    how='left'
)
machinery_goods_numerical_metrics_df['category'] = ''
machinery_goods_numerical_metrics_df.sort_values(by=['pillar', 'metric_name']).to_csv('machinery_goods_metric_info.csv', index=False)


## This part will export a csv for all industry and an excel checklist, only the final result will be store in github

In [80]:
df['industry_name'].drop_duplicates().sort_values().to_excel('industry_mapping_checklist.xlsx', index=False)

In [84]:
industry_list = df['industry_name'].drop_duplicates().tolist()
for industry in industry_list:
    industry_df = df[df['industry_name'] == industry]
    numerical_metrics = industry_df[industry_df['metric_unit'] != 'Yes/No']
    numerical_metrics_unique = numerical_metrics[['metric_name', 'metric_description', 'metric_unit', 'data_type', 'pillar']].drop_duplicates()
    numerical_metrics_df = numerical_metrics_unique.merge(
        updated_metric_info[['metric_name', 'metric_name_update']],
        on='metric_name',
        how='left'
    )
    numerical_metrics_df['category'] = ''
    numerical_metrics_df.sort_values(by=['pillar', 'metric_name']).to_csv(f'industry_before/{industry}_metric_info.csv', index=False)

### This part will combine all done mapped industry csv to single csv

In [26]:
#SASB mapping table to csv

mapped_files = os.listdir("../ingestion_service/data/industry_with_categories/")
mapped_csv_files = [f for f in mapped_files if re.search(r'.*info\.csv$', f)]
dfs = []
for csv in mapped_csv_files:
    industry_name = csv[:-16]
    df_chunk = pd.read_csv(f"../ingestion_service/data/industry_with_categories/{csv}", sep=',')
    df_chunk = df_chunk.dropna(subset=['category'])
    df_chunk['industry_name'] = industry_name
    dfs.append(df_chunk)

df = pd.concat(dfs, ignore_index=True)
df

,metric_name,metric_description,metric_unit,data_type,pillar,metric_name_update,category,industry_name
0,CO2DIRECTSCOPE1,Scope 1 emissions that occur within a company’...,Tons CO2e,float,E,Direct CO2 emissions scope 1,Product Design & Lifecycle Management,Commercial Banks
1,CO2INDIRECTSCOPE2,Scope 2 emissions that result from the generat...,Tons CO2e,float,E,Indirect CO2 emissions scope 2,Product Design & Lifecycle Management,Commercial Banks
2,CO2INDIRECTSCOPE3,Scope 3 emissions that occur in the value chai...,Tons CO2e,float,E,Indirect CO2 emissions scope 3,Product Design & Lifecycle Management,Commercial Banks
3,ANALYTICWASTERECYCLINGRATIO,Total recycled and reused waste produced in to...,%,float,E,Recycled and reused waste %,Waste & Hazardous Materials Management,Auto Parts
4,ENERGYUSETOTAL,Total energy consumed by a company within its ...,GJ,float,E,Total energy used,Energy Management,Auto Parts
...,...,...,...,...,...,...,...,...
83,ENERGYUSETOTAL,Total energy consumed by a company within its ...,GJ,float,E,Total energy used,Energy Management,Semiconductors
84,HAZARDOUSWASTE,Total amount of hazardous waste produced in tons.,Tons,float,E,Total hazardous waste,Waste & Hazardous Materials Management,Semiconductors
85,TRANALYTICRENEWENERGYUSE,Total energy consumed from primary renewable e...,%,float,E,Renewable energy use %,Energy Management,Semiconductors
86,WATERWITHDRAWALTOTAL,Total water withdrawal in cubic meters.,Cubic meters,float,E,Total water withdrawal,Water & Wastewater Management,Semiconductors


In [28]:
sasb_mapping_df = df[['industry_name', 'category', 'metric_name', 'metric_name_update']]
sasb_mapping_df.rename(columns={
    'industry_name': 'industry_name',
    'category': 'category',
    'metric_name': 'metric_name',
    'metric_name_update': 'updated_metric_name'
}, inplace=True)
sasb_mapping_df

/var/folders/pt/8p9w36290ms15w_t5l6pm9sr0000gn/T/ipykernel_16153/1873071653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sasb_mapping_df.rename(columns={


,industry_name,category,metric_name,updated_metric_name
0,Commercial Banks,Product Design & Lifecycle Management,CO2DIRECTSCOPE1,Direct CO2 emissions scope 1
1,Commercial Banks,Product Design & Lifecycle Management,CO2INDIRECTSCOPE2,Indirect CO2 emissions scope 2
2,Commercial Banks,Product Design & Lifecycle Management,CO2INDIRECTSCOPE3,Indirect CO2 emissions scope 3
3,Auto Parts,Waste & Hazardous Materials Management,ANALYTICWASTERECYCLINGRATIO,Recycled and reused waste %
4,Auto Parts,Energy Management,ENERGYUSETOTAL,Total energy used
...,...,...,...,...
83,Semiconductors,Energy Management,ENERGYUSETOTAL,Total energy used
84,Semiconductors,Waste & Hazardous Materials Management,HAZARDOUSWASTE,Total hazardous waste
85,Semiconductors,Energy Management,TRANALYTICRENEWENERGYUSE,Renewable energy use %
86,Semiconductors,Water & Wastewater Management,WATERWITHDRAWALTOTAL,Total water withdrawal


In [30]:
sasb_mapping_df.to_csv("../ingestion_service/data/industry_with_categories/sasb_mapping_table.csv", index=False)